In [1]:
import sage.logic.propcalc as pc
import itertools as it
import numpy.random as rn
from sage.plot.histogram import Histogram
import numpy as np

In [2]:
# Define the Boolean Rules
Rules = """E2F1 *= (~pRB & E2F1) | (~pRB & Myc)
CyclinD1 *= (~p21 & Myc) | (~pRB & Myc & E2F1) | (~p21 & E2F1) | (~pRB & E2F1 & CyclinD1)
p21 *= ~CyclinE
Myc *= ~pRB & E2F1
CyclinE *= ~pRB & ~p27Kip1 & E2F1
pRB *= (p27Kip1 & ~CyclinD1) | (~CyclinE & ~CyclinD1)
p27Kip1 *= ~CyclinE & ~CyclinD1"""

In [3]:
# Parse the Boolean Rules
rdict = {}
for line in Rules.split('\n'):
    lr = line.split('*=')
    rdict[lr[0].strip()] = pc.formula(lr[1])
    
for r in rdict: print r, "*=", rdict[r]

E2F1 *= (~pRB&E2F1)|(~pRB&Myc)
CyclinE *= ~pRB&~p27Kip1&E2F1
pRB *= (p27Kip1&~CyclinD1)|(~CyclinE&~CyclinD1)
p27Kip1 *= ~CyclinE&~CyclinD1
p21 *= ~CyclinE
CyclinD1 *= (~p21&Myc)|(~pRB&Myc&E2F1)|(~p21&E2F1)|(~pRB&E2F1&CyclinD1)
Myc *= ~pRB&E2F1


In [4]:
# Convert the rules to Hill kinetics rate functions
x={r:var('x_'+r) for r in rdict}
RF = {}
for r in rdict:
    k = {}
    n = {}
    H = {}
    
    RF[r] = -x[r]
    
    varnames = str(rdict[r].truthtable(end=0)).split()[0:-1]
    
    for ncomb in range(len(varnames)+1):
        for comb in it.combinations(varnames,ncomb):
            st = {v: v in list(comb) for v in varnames}
            hprod = 0
            if rdict[r].evaluate(st):           
                hprod = 1
                for v in varnames:
                    rv = r + '_' + v
                    k[rv] = var('k_' + rv)
                    n[rv] = var('n_' + rv)
                    
                    if st[v]:
                        hprod *= x[v]^n[rv] / (k[rv]^n[rv] + x[v]^n[rv])
                    else:
                        hprod *= k[rv]^n[rv] / (k[rv]^n[rv] + x[v]^n[rv])
            RF[r] += hprod
    RF[r] = ((RF[r]+x[r]).numerator()/(RF[r]+x[r]).denominator()) - x[r]

In [5]:
# Function to help obtain the "worst case system" of Rozum & Albert 2018 (PLOS Comp. Bio)
# Given v and lists vmin and vmax, returns the rate function for v with the variables in vmin set to zero
# and variables in vmax set to infinity
def wcRF(v, vmin, vmax):
    vmin = set([x[vmx] for vmx in vmin])
    vmax = set([x[vmx] for vmx in vmax])
    mn1 = 0
    md1 = 0
    
    if (RF[v]+x[v]).numerator().operator() == sage.symbolic.operators.mul_vararg:
        numerTerms = [(RF[v]+x[v]).numerator()]
    else:
        numerTerms = (RF[v]+x[v]).numerator().operands()
    
    for t in numerTerms:
        if vmin.isdisjoint(set(t.args())) and vmax.issubset(set(t.args())):
            for vmx in vmax:
                t = t.subs(vmx == 1)
            mn1 += t
    for t in (RF[v]+x[v]).denominator().expand().operands():
        for vmx in vmin:
            t = t.subs(vmx == 0)
        if vmax.issubset(set(t.args())):
            for vmx in vmax:
                t = t.subs(vmx == 1)
            md1 += t
    #for t in (RF[v]+x[v]).denominator().operands():
    #    if vmax.isdisjoint(set(t.args())):
    #        md1 *= t
    return simplify(mn1/md1 - x[v])

In [6]:
# The worst case system for this motif
wcCycE=RF["CyclinE"]
wcp27=wcRF("p27Kip1",["CyclinD1"],[])
wcpRB=wcRF("pRB",["CyclinD1"],[])
wcE2F1=wcRF("E2F1",["Myc"],[])

In [7]:
show(wcCycE)
show(wcp27)
show(wcpRB)
show(wcE2F1)

-x_CyclinE + k_CyclinE_p27Kip1^n_CyclinE_p27Kip1*k_CyclinE_pRB^n_CyclinE_pRB*x_E2F1^n_CyclinE_E2F1/((k_CyclinE_E2F1^n_CyclinE_E2F1 + x_E2F1^n_CyclinE_E2F1)*(k_CyclinE_p27Kip1^n_CyclinE_p27Kip1 + x_p27Kip1^n_CyclinE_p27Kip1)*(k_CyclinE_pRB^n_CyclinE_pRB + x_pRB^n_CyclinE_pRB))

k_p27Kip1_CyclinD1^n_p27Kip1_CyclinD1*k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(k_p27Kip1_CyclinD1^n_p27Kip1_CyclinD1*k_p27Kip1_CyclinE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinD1^n_p27Kip1_CyclinD1*x_CyclinE^n_p27Kip1_CyclinE) - x_p27Kip1

-x_pRB + (k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*x_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinD1^n_pRB_CyclinD1*x_CyclinE^n_pRB_CyclinE*x_p27Kip1^n_pRB_p27Kip1)/(k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_p27Kip1^n_pRB_p27Kip1*x_CyclinE^n_pRB_CyclinE + k_pRB_CyclinD1^n_pRB_CyclinD1*k_pRB_CyclinE^n_pRB_CyclinE*x_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinD1^n_pRB_CyclinD1*x_CyclinE^n_pRB_CyclinE*x_p27Kip1^n_pRB_p27Kip1)

k_E2F1_Myc^n_E2F1_Myc*k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1/(k_E2F1_E2F1^n_E2F1_E2F1*k_E2F1_Myc^n_E2F1_Myc*k_E2F1_pRB^n_E2F1_pRB + k_E2F1_Myc^n_E2F1_Myc*k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1 + k_E2F1_E2F1^n_E2F1_E2F1*k_E2F1_Myc^n_E2F1_Myc*x_pRB^n_E2F1_pRB + k_E2F1_Myc^n_E2F1_Myc*x_E2F1^n_E2F1_E2F1*x_pRB^n_E2F1_pRB) - x_E2F1

In [8]:
# Simplify the system at steady state
wcp27=(wcp27+x["p27Kip1"]).simplify_rational()-x["p27Kip1"]
wcpRB=(wcpRB+x["pRB"]).simplify_rational()-x["pRB"]
wcE2F1=(wcE2F1+x["E2F1"]).simplify_rational()-x["E2F1"]
show(wcCycE)
show(wcp27)
show(wcpRB)
show(wcE2F1)

-x_CyclinE + k_CyclinE_p27Kip1^n_CyclinE_p27Kip1*k_CyclinE_pRB^n_CyclinE_pRB*x_E2F1^n_CyclinE_E2F1/((k_CyclinE_E2F1^n_CyclinE_E2F1 + x_E2F1^n_CyclinE_E2F1)*(k_CyclinE_p27Kip1^n_CyclinE_p27Kip1 + x_p27Kip1^n_CyclinE_p27Kip1)*(k_CyclinE_pRB^n_CyclinE_pRB + x_pRB^n_CyclinE_pRB))

-x_p27Kip1 + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(k_p27Kip1_CyclinE^n_p27Kip1_CyclinE + x_CyclinE^n_p27Kip1_CyclinE)

-x_pRB + (k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + (k_pRB_CyclinE^n_pRB_CyclinE + x_CyclinE^n_pRB_CyclinE)*x_p27Kip1^n_pRB_p27Kip1)/(k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_p27Kip1^n_pRB_p27Kip1*x_CyclinE^n_pRB_CyclinE + (k_pRB_CyclinE^n_pRB_CyclinE + x_CyclinE^n_pRB_CyclinE)*x_p27Kip1^n_pRB_p27Kip1)

k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1/(k_E2F1_E2F1^n_E2F1_E2F1*k_E2F1_pRB^n_E2F1_pRB + k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1 + (k_E2F1_E2F1^n_E2F1_E2F1 + x_E2F1^n_E2F1_E2F1)*x_pRB^n_E2F1_pRB) - x_E2F1

In [9]:
# Define a threshold variable; we will study CyclinE > TE
var('TE')

TE

In [10]:
# A functional stable motif is obtained when TPRB-TE=pE2F1=0 has a solution in (0,1)
TP27=wcp27.subs(x["CyclinE"]==TE).solve(x["p27Kip1"])[0].rhs()
TPRB=wcpRB.subs(x["CyclinE"]==TE,x["p27Kip1"]==TP27).solve(x["pRB"])[0].rhs()
pE2F1=wcE2F1.subs(x["pRB"]==TPRB)

In [11]:
show(TP27)
show(TPRB)
show(pE2F1)

k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(TE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE)

(k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + (TE^n_pRB_CyclinE + k_pRB_CyclinE^n_pRB_CyclinE)*(k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(TE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE))^n_pRB_p27Kip1)/(TE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + (TE^n_pRB_CyclinE + k_pRB_CyclinE^n_pRB_CyclinE)*(k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(TE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE))^n_pRB_p27Kip1)

k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1/(k_E2F1_E2F1^n_E2F1_E2F1*k_E2F1_pRB^n_E2F1_pRB + k_E2F1_pRB^n_E2F1_pRB*x_E2F1^n_E2F1_E2F1 + (k_E2F1_E2F1^n_E2F1_E2F1 + x_E2F1^n_E2F1_E2F1)*((k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + (TE^n_pRB_CyclinE + k_pRB_CyclinE^n_pRB_CyclinE)*(k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(TE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE))^n_pRB_p27Kip1)/(TE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + k_pRB_CyclinE^n_pRB_CyclinE*k_pRB_p27Kip1^n_pRB_p27Kip1 + (TE^n_pRB_CyclinE + k_pRB_CyclinE^n_pRB_CyclinE)*(k_p27Kip1_CyclinE^n_p27Kip1_CyclinE/(TE^n_p27Kip1_CyclinE + k_p27Kip1_CyclinE^n_p27Kip1_CyclinE))^n_pRB_p27Kip1))^n_E2F1_pRB) - x_E2F1

In [12]:
# Function to search for roots of f on an interval [a,b] with tolerance eps
def find_all_roots(f, a, b, eps=0.001):
    roots = []
    intervals_to_check = [(a,b)]
    while intervals_to_check:
        start, end = intervals_to_check.pop()
        try:
            root = find_root(f, start, end)
        except RuntimeError:
            continue
        if root in roots:
            continue
        if abs(f(root)) < 1:
            roots.append(root)
        intervals_to_check.extend([(start, root-eps), (root+eps, end)])
    roots.sort()
    return roots

# Find the midpoint of the line segment connecting (x1,y1) to (x2,y2)
def linInterpRoot(x1,x2,y1,y2):
    m = (y2-y1)/(x2-x1)
    b = y1-x1*m
    x0 = -b/m
    return x0

# Function to check if a given root of TPRB-TE allows a solution to pE2F1=0
# Note: lack of roots returned does not mean no roots anywhere, just between 0.5 an 1
def testTE( tval, adict, lowroot, highroot, eps ):
    kappaTE = []
    rlist = find_all_roots( pE2F1.subs(adict).subs(TE==tval), lowroot, highroot, eps)
    for r in rlist:
        if r == 0: continue
        # Dt(CycE) + CycE is val that CycE takes, then subtract off CycE again and compare to zero
        kappaTE.append( (r,wcCycE.subs(x["pRB"]==TPRB,x["p27Kip1"]==TP27,x["E2F1"]==r,x["CyclinE"]==TE).subs(adict).subs(TE==tval)) )
    return kappaTE
    
# Class to store parameter sets and test them for functionality        
class SMRoot:
    Nscan = 20
    tscan = [i/Nscan for i in range(1,Nscan+1)]
    lowroot = 0.499
    highroot = 1
    eps = 0.001

    def __init__(self,paramDict):
        self.paramDict=paramDict
        self.maxRootDict = {}
        self.tested = False
        
    def buildMaxRoot(self):
        kappaOld = None
        tOld = SMRoot.tscan[0]
        for t in SMRoot.tscan:
            kappaTE = testTE( t, self.paramDict, SMRoot.lowroot, SMRoot.highroot, SMRoot.eps )
            
            if len(kappaTE) > 0:
                kappaTE = max(kappaTE,key=lambda xx: xx[0])
            else:
                kappaOld = None
                tOld = t
                continue

            # because we scan forwards, we only ever overwrite with bigger roots
            if kappaOld != None and (bool(kappaOld < 0) !=  bool(kappaTE[1] < 0)):
                TERoot = linInterpRoot(tOld,t,kappaOld, kappaTE[1])
                self.maxRootDict = {"CyclinE":TERoot,
                 "p27":TP27.subs(self.paramDict).subs(TE=TERoot),
                 "pRB":TPRB.subs(self.paramDict).subs(TE=TERoot),
                 "E2F1":kappaTE[0]}
            kappaOld = kappaTE[1] 
       
        self.tested = True

In [13]:
# Read unspecified parameters and build a function to randomly generate values for them
kargs = []
nargs = []
for v in pE2F1.args() + wcCycE.args():
    if str(v)[0]=="k":
        kargs.append(v)
    if str(v)[0]=="n":
        nargs.append(v)

kargs = list(set(kargs))
nargs = list(set(nargs))
nkargs = nargs + kargs
   

def genRandomParams(seed = None, scale = "Log", nValBounds = (1,8),kValBounds = (1/16,15/16)):
    rn.seed(seed)
    
    # sample low values exponentially more often than high values (uniform in log-space); otherwise, uniform in linear-space
    if scale == "Log":
        nvals = [-ln(xx) for xx in rn.uniform(exp(-nValBounds[0]),exp(-nValBounds[1]),len(nargs))]
        kvals = [-ln(xx) for xx in rn.uniform(exp(-kValBounds[0]),exp(-kValBounds[1]),len(kargs))]
    else:
        nvals = [xx for xx in rn.uniform(nValBounds[0],nValBounds[1],len(nargs))]
        kvals = [xx for xx in rn.uniform(kValBounds[0],kValBounds[1],len(kargs))]
        
    nkvals = nvals + kvals
    return {nkargs[i]:nkvals[i] for i in range(len(nkvals))}

In [14]:
testDict=genRandomParams(nValBounds=(3,3),kValBounds=(3/8,3/8))
testRoot = SMRoot(testDict)
testRoot.buildMaxRoot()
print(testRoot.maxRootDict)

{'p27': 0.09075895948227916, 'E2F1': 0.9353527916974319, 'CyclinE': 0.8084027367312884, 'pRB': 0.10346880081760973}


/usr/lib/sagemath/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:27: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.


In [ ]:
# Test functionality for NSamples parameter sets for this candidate motif
NSamples = 10000

SM = []
NSM = []
strongSM = []
for i in range(NSamples):
    testDict=genRandomParams(seed=i,scale="Lin")
    testRoot = SMRoot(testDict)
    testRoot.buildMaxRoot()
    if len(testRoot.maxRootDict.keys()) > 0:
        SM.append(testRoot)
        r = testRoot.maxRootDict
        if r["pRB"] < 0.5 and r["p27"] < 0.5 and r["CyclinE"] > 0.5 and r["E2F1"] > 0.5:
            strongSM.append(testRoot)
    else:
        NSM.append(testRoot)


In [ ]:
# Save output
save(SM,'SM1')
save(strongSM,'strongSM1')
save(NSM,'NSM1')

In [15]:
# Load output if it exists
NSM = load('./NSM1.sobj')
SM = load('./SM1.sobj')
strongSM = load('./strongSM1.sobj')

In [19]:
# Some basic statistics
print(float(len(SM))/(len(NSM) + len(SM)))
print(float(len(strongSM))/(len(SM)))
print(float(len(strongSM))/(len(NSM) + len(SM)))

0.1721
0.818710052295
0.1409
